# Analyse des données physiques avec MLP

Ce notebook implémente la classification par Perceptron Multi-Couches (MLP) sur notre jeu de données physiques pour la détection d'attaques. Nous évaluerons les performances à la fois sur la classification binaire (attaque vs normal) et la classification multiclasse (types d'attaques spécifiques).

In [ ]:
# Libraries
import pandas as pd
import numpy as np
from pickleshare import PickleShareDB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score,
    confusion_matrix, matthews_corrcoef, balanced_accuracy_score
)
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import tracemalloc
import os

## Chargement des données

In [ ]:
# Load data from prep_data
data_dir = '../prep_data' 
db = PickleShareDB(os.path.join(data_dir, 'kity'))

# Load raw data
df_phy_1 = db['df_phy_1']
df_phy_2 = db['df_phy_2']
df_phy_3 = db['df_phy_3']
df_phy_4 = db['df_phy_4']
df_phy_norm = db['df_phy_norm']

# Load label mapping
label_mapping = db['label_mapping']

## Préparation des données

Pour MLP, nous devons préparer soigneusement nos données :

- Supprimer la colonne Time car l'information temporelle n'est pas directement utile pour la classification
- Garder les mesures des capteurs de réservoir et de débit comme indicateurs physiques clés
- Garder les états des vannes et des pompes car ils représentent les actions du système
- Mettre à l'échelle les caractéristiques numériques car les MLP sont sensibles à la mise à l'échelle des caractéristiques
- Encoder en one-hot les variables catégorielles
- Supprimer toutes les colonnes constantes ou redondantes identifiées lors d'analyses précédentes

In [ ]:
def prepare_data(df):
    """Prepare data with proper handling of missing values"""
    # Remove Time column
    df_prepared = df.drop(columns=['Time'])
    
    # Split features and labels
    X = df_prepared.drop(columns=['Label', 'Label_n'])
    y_label = df_prepared['Label']
    y_label_n = df_prepared['Label_n']
    
    return X, y_label, y_label_n

def preprocess_data(df):
    """Preprocess data with imputation and scaling"""
    # Separate numeric and categorical columns
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = df.select_dtypes(include=['category', 'bool']).columns
    
    # Handle numeric features
    if len(numeric_cols) > 0:
        num_imputer = SimpleImputer(strategy='mean')
        scaler = StandardScaler()
        df[numeric_cols] = num_imputer.fit_transform(df[numeric_cols])
        df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
    
    # Handle categorical features
    if len(categorical_cols) > 0:
        cat_imputer = SimpleImputer(strategy='most_frequent')
        df[categorical_cols] = cat_imputer.fit_transform(df[categorical_cols])
        df = pd.get_dummies(df, columns=categorical_cols)
    
    return df

# Prepare datasets
print("Preparing datasets...")
prepared_data = {}
for name, df in {
    'phy_1': df_phy_1,
    'phy_2': df_phy_2,
    'phy_3': df_phy_3,
    'phy_4': df_phy_4,
    'phy_norm': df_phy_norm
}.items():
    X, y_label, y_label_n = prepare_data(df)
    X_processed = preprocess_data(X)
    prepared_data[name] = {
        'X': X_processed,
        'y_label': y_label,
        'y_label_n': y_label_n
    }

# Combine datasets
X_all = pd.concat([data['X'] for data in prepared_data.values()])
y_label_all = pd.concat([data['y_label'] for data in prepared_data.values()])
y_label_n_all = pd.concat([data['y_label_n'] for data in prepared_data.values()])

print("Data preparation complete.")
print("X_all shape:", X_all.shape)

## Implémentation et fonctions d'évaluation MLP

In [ ]:
def train_evaluate_mlp(X_train, X_test, y_train, y_test):
    """Train and evaluate MLP model with performance metrics"""
    print("NaN check in train_evaluate_mlp:")
    print("X_train NaNs:", X_train.isna().sum().sum())
    print("X_test NaNs:", X_test.isna().sum().sum())
    
    # Handle any remaining NaN values if needed
    if X_train.isna().sum().sum() > 0 or X_test.isna().sum().sum() > 0:
        imputer = SimpleImputer(strategy='mean')
        X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
        X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)
    
    mlp = MLPClassifier(
        hidden_layer_sizes=(100, 50),  # Two hidden layers
        activation='relu',
        solver='adam',
        max_iter=300,
        random_state=42
    )
    
    # Measure training time and memory
    tracemalloc.start()
    start_fit_time = time.time()
    
    mlp.fit(X_train, y_train)
    
    fit_time = time.time() - start_fit_time
    current, peak = tracemalloc.get_traced_memory()
    fit_memory_usage = peak / (1024 * 1024)  # Convert to MB
    tracemalloc.stop()
    
    # Measure prediction time and memory
    tracemalloc.start()
    start_predict_time = time.time()
    
    y_pred = mlp.predict(X_test)
    
    predict_time = time.time() - start_predict_time
    current, peak = tracemalloc.get_traced_memory()
    predict_memory_usage = peak / (1024 * 1024)  # Convert to MB
    tracemalloc.stop()
    
    # Calculate metrics
    conf_matrix = confusion_matrix(y_test, y_pred)
    TN, FP, FN, TP = conf_matrix.ravel()
    
    return {
        'confusion_matrix': conf_matrix,
        'TP': TP,
        'FP': FP,
        'TN': TN,
        'FN': FN,
        'precision': precision_score(y_test, y_pred, average='binary'),
        'recall': recall_score(y_test, y_pred, average='binary'),
        'tnr': TN / (TN + FP) if (TN + FP) != 0 else 0,
        'fpr': FP / (FP + TN) if (FP + TN) != 0 else 0,
        'accuracy': accuracy_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred, average='binary'),
        'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
        'mcc': matthews_corrcoef(y_test, y_pred),
        'fit_time': fit_time,
        'predict_time': predict_time,
        'fit_memory_usage': fit_memory_usage,
        'predict_memory_usage': predict_memory_usage
    }

## Entraînement et évaluation du modèle - Classification binaire

In [ ]:
# Split data for binary classification
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_label_n_all, test_size=0.2, random_state=42
)

# Train and evaluate model
binary_results = train_evaluate_mlp(X_train, X_test, y_train, y_test)

# Add metadata
binary_results['data'] = 'PHY'
binary_results['model_type'] = 'mlp'
binary_results['attack_type'] = 'labeln'

## Visualiser les résultats de la classification binaire

In [ ]:
# Plot confusion matrix
fig = go.Figure(data=go.Heatmap(
    z=binary_results['confusion_matrix'],
    x=['Predicted Negative', 'Predicted Positive'],
    y=['Actual Negative', 'Actual Positive'],
    text=binary_results['confusion_matrix'],
    texttemplate="%{text}",
    textfont={"size": 16},
    colorscale='Blues'
))

fig.update_layout(
    title='Confusion Matrix - Binary Classification',
    height=400,
    width=500
)

fig.show()

## Classification multiclasse

In [ ]:
# Encode labels
le = LabelEncoder()
le.classes_ = np.array(list(label_mapping.keys()))
y_label_all_encoded = le.transform(y_label_all)

# Dictionary to store results for each attack type
multiclass_results = {}

# Train and evaluate for each attack type
for attack_label, encoded_label in label_mapping.items():
    if attack_label != 'normal':  # Skip normal class as it's our reference
        print(f"\nProcessing attack type: {attack_label}")
        
        # Create binary labels for this attack type
        y_binary = (y_label_all_encoded == encoded_label).astype(int)
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X_all, y_binary, test_size=0.2, random_state=42
        )
        
        results = train_evaluate_mlp(X_train, X_test, y_train, y_test)
        results['data'] = 'PHY'
        results['model_type'] = 'mlp'
        results['attack_type'] = attack_label
        multiclass_results[attack_label] = results

## Visualiser les résultats de la classification multiclasse

In [ ]:
def plot_attack_metrics(metric_name):
    attacks = list(multiclass_results.keys())
    values = [results[metric_name] for results in multiclass_results.values()]
    
    fig = go.Figure(data=[go.Bar(x=attacks, y=values)])
    
    fig.update_layout(
        title=f'{metric_name} by Attack Type',
        yaxis_title='Score',
        xaxis_title='Attack Type'
    )
    return fig

# Plot key metrics
metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1']
for metric in metrics_to_plot:
    plot_attack_metrics(metric).show()

In [ ]:
# Plot confusion matrices for each attack type
for attack_type, results in multiclass_results.items():
    fig = go.Figure(data=go.Heatmap(
        z=results['confusion_matrix'],
        x=['Predicted Negative', 'Predicted Positive'],
        y=['Actual Negative', 'Actual Positive'],
        text=results['confusion_matrix'],
        texttemplate="%{text}",
        textfont={"size": 16},
        colorscale='Blues'
    ))
    
    fig.update_layout(
        title=f'Confusion Matrix - {attack_type}',
        height=400,
        width=500
    )
    
    fig.show()

In [ ]:
# Create summary table
summary_data = []

for attack_type, results in multiclass_results.items():
    summary_data.append({
        'Attack Type': attack_type,
        'Accuracy': results['accuracy'],
        'Precision': results['precision'],
        'Recall': results['recall'],
        'F1 Score': results['f1'],
        'Training Time (s)': results['fit_time'],
        'Prediction Time (s)': results['predict_time'],
        'Training Memory (MB)': results['fit_memory_usage'],
        'Prediction Memory (MB)': results['predict_memory_usage']
    })

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.round(4)
display(summary_df)

## Analyse des performances de calcul

In [ ]:
# Plot computational performance metrics
performance_fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Training and Prediction Time', 'Memory Usage')
)

# Time metrics
attacks = list(multiclass_results.keys())
train_times = [results['fit_time'] for results in multiclass_results.values()]
predict_times = [results['predict_time'] for results in multiclass_results.values()]

performance_fig.add_trace(
    go.Bar(name='Training Time', x=attacks, y=train_times),
    row=1, col=1
)
performance_fig.add_trace(
    go.Bar(name='Prediction Time', x=attacks, y=predict_times),
    row=1, col=1
)

# Memory metrics
train_memory = [results['fit_memory_usage'] for results in multiclass_results.values()]
predict_memory = [results['predict_memory_usage'] for results in multiclass_results.values()]

performance_fig.add_trace(
    go.Bar(name='Training Memory', x=attacks, y=train_memory),
    row=2, col=1
)
performance_fig.add_trace(
    go.Bar(name='Prediction Memory', x=attacks, y=predict_memory),
    row=2, col=1
)

performance_fig.update_layout(
    height=800,
    title_text="Computational Performance by Attack Type",
    barmode='group'
)

performance_fig.show()

## Sauvegarder les résultats pour Streamlit

In [ ]:
# Save binary classification results
db['PHY_results_mlp_labeln'] = binary_results

# Save multiclass results
for attack_label in multiclass_results.keys():
    db[f'PHY_results_mlp_{attack_label}'] = multiclass_results[attack_label]

# Save summary statistics
db['mlp_summary_stats'] = {
    'summary_df': summary_df,
    'binary_results': binary_results,
    'multiclass_results': multiclass_results
}